# Assignment (Quiz 6)

In [1]:
import graphlab

In [2]:
people = graphlab.SFrame('people_wiki.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v1.10.1 started. Logging: /tmp/graphlab_server_1467022766.log


This non-commercial license of GraphLab Create is assigned to 1200259@eng.asu.edu.eg and will expire on June 24, 2017. For commercial licensing options, visit https://dato.com/buy/.


## create word_count and tfidf columns for all of the documents

In [3]:
people['word_count'] = graphlab.text_analytics.count_words(people['text'])

In [4]:
people['tfidf'] = graphlab.text_analytics.tf_idf(people['word_count'])

* <B>Compare top words according to word counts to TF-IDF:</B>

In [5]:
elton_john = people[people['name'] == 'Elton John']

In [6]:
elton_word_count_table = elton_john[['word_count']].stack('word_count', new_column_name = ['word','count']).sort('count', ascending=False)
elton_word_count_table

word,count
the,27
in,18
and,15
of,13
a,10
has,9
he,7
john,7
on,6
since,5


### 1- Top word count words for Elton John

In [7]:
elton_word_count_table[0:3]['word']

dtype: str
Rows: 3
['the', 'in', 'and']

In [8]:
elton_tfidf_table = elton_john[['tfidf']].stack('tfidf', new_column_name = ['word','tfidf']).sort('tfidf', ascending=False)
elton_tfidf_table

word,tfidf
furnish,18.38947184
elton,17.48232027
billboard,17.3036809575
john,13.9393127924
songwriters,11.250406447
overallelton,10.9864953892
tonightcandle,10.9864953892
19702000,10.2933482087
fivedecade,10.2933482087
aids,10.262846934


### 2- Top TF-IDF words for Elton John

In [9]:
elton_tfidf_table[0:3]['word']

dtype: str
Rows: 3
['furnish', 'elton', 'billboard']

* <B>Measuring distance:</B>

In [10]:
victoria_beckham = people[people['name'] == 'Victoria Beckham']

In [11]:
paul_mccartney = people[people['name'] == 'Paul McCartney']

### 3- The cosine distance between 'Elton John's and 'Victoria Beckham's articles (represented with TF-IDF) falls within which range?

In [12]:
graphlab.distances.cosine(elton_john['tfidf'][0], victoria_beckham['tfidf'][0])

0.9567006376655429

### 4- The cosine distance between 'Elton John's and 'Paul McCartney's articles (represented with TF-IDF) falls within which range?

In [13]:
graphlab.distances.cosine(elton_john['tfidf'][0], paul_mccartney['tfidf'][0])

0.8250310029221779

### 5- Who is closer to 'Elton John', 'Victoria Beckham' or 'Paul McCartney'?
    Paul McCartney

* <B>Building nearest neighbors models with different input features and setting the distance metric:</B>

In [14]:
knn_count_model = graphlab.nearest_neighbors.create(people, features=['word_count'], label='name', distance='cosine')

Starting brute force nearest neighbors model training.

In [15]:
knn_tfidf_model = graphlab.nearest_neighbors.create(people, features=['tfidf'], label='name', distance='cosine')

Starting brute force nearest neighbors model training.

In [16]:
elton_count_query = knn_count_model.query(elton_john)
elton_count_query

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 39.885ms     |

| Done         |         | 100         | 772.274ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,2.22044604925e-16,1
0,Cliff Richard,0.16142415259,2
0,Sandro Petrone,0.16822542751,3
0,Rod Stewart,0.168327165587,4
0,Malachi O'Doherty,0.177315545979,5


### 6- Who is the nearest neighbor to 'Elton John' using raw word counts?

In [17]:
elton_count_query[1]['reference_label']

'Cliff Richard'

In [18]:
elton_tfidf_query = knn_tfidf_model.query(elton_john)
elton_tfidf_query

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 31.032ms     |

| Done         |         | 100         | 808.459ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Elton John,-2.22044604925e-16,1
0,Rod Stewart,0.717219667893,2
0,George Michael,0.747600998969,3
0,Sting (musician),0.747671954431,4
0,Phil Collins,0.75119324879,5


### 7- Who is the nearest neighbor to 'Elton John' using TF-IDF?

In [19]:
elton_tfidf_query[1]['reference_label']

'Rod Stewart'

In [20]:
victoria_count_query = knn_count_model.query(victoria_beckham)
victoria_count_query

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 13.451ms     |

| Done         |         | 100         | 594.719ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,-2.22044604925e-16,1
0,Mary Fitzgerald (artist),0.207307036115,2
0,Adrienne Corri,0.214509782788,3
0,Beverly Jane Fry,0.217466468741,4
0,Raman Mundair,0.217695474992,5


### 8- Who is the nearest neighbor to 'Victoria Beckham' using raw word counts?

In [21]:
victoria_count_query[1]['reference_label']

'Mary Fitzgerald (artist)'

In [22]:
victoria_tfidf_query = knn_tfidf_model.query(victoria_beckham)
victoria_tfidf_query

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.00169288  | 16.871ms     |

| Done         |         | 100         | 727.335ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,Victoria Beckham,1.11022302463e-16,1
0,David Beckham,0.548169610263,2
0,Stephen Dow Beckham,0.784986706828,3
0,Mel B,0.809585523409,4
0,Caroline Rush,0.819826422919,5


### 9- Who is the nearest neighbor to 'Victoria Beckham' using TF-IDF?

In [23]:
victoria_tfidf_query[1]['reference_label']

'David Beckham'